In [1]:
import numpy as np
import matplotlib.pyplot as plt
import joblib

In [2]:
eeg = joblib.load('altered_data/merged_eeg.sav')
print(eeg['X'].shape, len(eeg['y']))

(3945, 8, 1250) 3945


#### STFT

In [3]:
import scipy
import scipy.signal as signal
from stft import *

d:\miniconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
eegSTFT = STFT(default_stft_params)
eegSTFT.compute(eeg['X'])
eeg['STFT']=eegSTFT.stft['Zxx']
print(eeg['STFT'].shape, eeg['X'].shape)

(3945, 8, 129, 11) (3945, 8, 1250)


#### wavelet transform

In [5]:
import pywt as pywt

def wavelet_coef_freq(signal, Fs=250, scale=(1,11), waveletname = 'cmor',method_='conv'):
  Ts=1/Fs
  scales=np.arange(scale[0],scale[1])
  f = pywt.scale2frequency(waveletname,scales)*Fs
  [coefficients, frequencies] = pywt.cwt(signal, scales, waveletname, 1/Fs, method=method_)
  return [coefficients, frequencies]

def _2D_wavelet_coef_freq(X, Fs=250, scale=(1,11), waveletname = 'cmor',method_='conv'):
  assert X.ndim == 2
  coeffs = np.empty((X.shape[0],scale[1]-scale[0],X.shape[1]))

  for ch in range(X.shape[0]):
    coeffs[ch,:] = wavelet_coef_freq(X[ch,:],Fs,scale,waveletname,method_)[0]

  return coeffs

def _3D_wavelet_coef_freq(X, Fs=250, scale=(1,11), waveletname = 'cmor',method_='conv'):
  assert X.ndim == 3
  
  coeffs = np.empty((X.shape[0],X.shape[1],scale[1]-scale[0],X.shape[2]))

  for ch in range(X.shape[0]):
    for i in range(X.shape[1]):
      coeffs[i,:,:] = _2D_wavelet_coef_freq(X[i,:,:], Fs, scale, waveletname, method_)
    print(ch)

  return coeffs

In [9]:
eeg['morlet_wavelet'] = _3D_wavelet_coef_freq(eeg['X'][0:50,:,:],250,(1,26),'morl','fft')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [10]:
print(eeg['morlet_wavelet'].shape)

(50, 8, 25, 1250)


In [12]:
joblib.dump(eeg,'features/morlet_wavelet.sav')

['features/morlet_wavelet.sav']

#### whatever it is you are trying to achieve here that you are not doing properly

In [15]:
# def plot_wavelet(coefficients, frequencies, cmap = plt.cm.seismic, 
#                  title = 'Wavelet Transform (Power Spectrum) of signal', 
#                  ylabel = 'Period (years)', 
#                  xlabel = 'Time'):
    
    
#     power = (abs(coefficients)) ** 2
#     period = 1. / frequencies
#     levels = [0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8]
#     contourlevels = np.log2(levels)
    
#     fig, ax = plt.subplots(figsize=(15, 10))
#     im = ax.contourf(time, np.log2(period), np.log2(power), contourlevels, extend='both',cmap=cmap)
    
#     ax.set_title(title, fontsize=20)
#     ax.set_ylabel(ylabel, fontsize=18)
#     ax.set_xlabel(xlabel, fontsize=18)
    
#     yticks = 2**np.arange(np.ceil(np.log2(period.min())), np.ceil(np.log2(period.max())))
#     ax.set_yticks(np.log2(yticks))
#     ax.set_yticklabels(yticks)
#     ax.invert_yaxis()
#     ylim = ax.get_ylim()
#     ax.set_ylim(ylim[0], -1)
    
#     cbar_ax = fig.add_axes([0.95, 0.5, 0.03, 0.25])
#     fig.colorbar(im, cax=cbar_ax, orientation="vertical")
#     plt.show()
 



In [ ]:
N = data.shape[0]
t0=1500
dt=0.25
time = np.arange(0, N) * dt + t0
signal = np.ndarray(data.shape[0])
 
scales = np.arange(1, 1250)
[coefficients, frequencies] = get_wavelet_coef_and_freq(time, signal, scales, waveletname='cmor')

#plot_wavelet(coefficients, frequencies)


/usr/local/lib/python3.7/dist-packages/pywt/_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should takethe form cmorB-C where B and C are floats representing the bandwidth frequency and center frequency, respectively (example: cmor1.5-1.0).
  wavelet = DiscreteContinuousWavelet(wavelet)


In [16]:
scales = range(1,1250)
waveletname = 'morl'
train_size = 10
test_size= 20

train_data_cwt = np.ndarray(shape=(train_size, 1249, 1249, 8))
f = pywt.scale2frequency(waveletname, scales)/1250

for ii in range(0,train_size):
    if ii % 100 == 0:
        print(ii)
    for jj in range(0,8):
        signal = d[ii, :, jj]
        coeff, freq = pywt.cwt(signal, scales, waveletname, 1)
        coeff_ = coeff[:,:1249]
        train_data_cwt[ii, :, :, jj] = coeff_

test_data_cwt = np.ndarray(shape=(test_size, 1249, 1249, 8))

for ii in range(0,test_size):
    if ii % 100 == 0:
        print(ii)
    for jj in range(0,8):
        signal = d[ii, :, jj]
        coeff, freq = pywt.cwt(signal, scales, waveletname, 1)
        coeff_ = coeff[:,:1249]
        test_data_cwt[ii, :, :, jj] = coeff_

labels_train = list(map(lambda x: int(x) - 1, np.ndarray(m.shape[0])))
labels_test = list(map(lambda x: int(x) - 1, np.ndarray(m.shape[0])))
x_train = train_data_cwt
y_train = list(labels_train[:train_size])
x_test = test_data_cwt
y_test = list(labels_test[:test_size])

0
0


In [ ]:
import keras
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import History 
history = History()
 
img_x = 1249
img_y = 1249
img_z = 8
input_shape = (img_x, img_y, img_z)
 
num_classes = 6
batch_size = 16
num_classes = 7
epochs = 10
 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
 
 
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
 
 
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[history])
 
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss: {}, Train accuracy: {}'.format(train_score[0], train_score[1]))
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy: {}'.format(test_score[0], test_score[1]))